In [ ]:
import sys
sys.path.append("..")

import io
import os
import random
import math
import time
import json
import shutil
from io import BytesIO
from pathlib import Path
from collections import OrderedDict
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from IPython.display import display

from src.datasets import *
from src.util.image import *
from src.util import *
from src.algo import *
from src.models.decoder import *
from src.models.transform import *
from src.models.util import *
from experiments import datasets
from experiments.denoise.resconv import ResConv
from src.algo.wangtiles import *

def resize(img, scale: float, mode: VF.InterpolationMode = VF.InterpolationMode.NEAREST):
    if isinstance(img, PIL.Image.Image):
        shape = (img.height, img.width)
    else:
        shape = img.shape[-2:]
    return VF.resize(img, [max(1, int(s * scale)) for s in shape], mode, antialias=False)

In [ ]:
tiles = WangTiles2E()
tiles

In [ ]:
template = tiles.create_template((32, 32))
template.image = VF.to_tensor(PIL.Image.open(
    #"/home/bergi/Pictures/wang-tiling-clipae-2dscroller-1-template.png"
    #"/home/bergi/Pictures/wang-tiling-clipae-ropes-1-template.png"
    #"/home/bergi/Pictures/wang-tiling-clipae-ropes-1-template.png"
    "/home/bergi/Pictures/wang-tiling-clipae-mountaintops-1-template.png"
))
print(template)
display(VF.to_pil_image(template.image))
print(template.indices)


In [ ]:
map = wang_map_stochastic_scanline(
    tiles,
    (32, 32),
    probabilities={0: 4, 1: 5, 11: .3},
)
VF.to_pil_image(template.render_map(map))